In [1]:
import sys
import os
import math
import random
import bisect
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import subprocess
import re
import tempfile
import itertools
import torch
import spacy
import amrlib
import penman
import openai

from typing import List, Tuple
from operator import itemgetter 
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, BertTokenizer, BertForSequenceClassification
# import qgrid

BASE_DIR = os.path.abspath(os.getcwd()+'/../..')  # /home/gil/dev/NEBULA2/
os.chdir(os.getcwd()+'/../..')
OPENAI_API_KEY=''
openai.api_key = OPENAI_API_KEY

from nebula_api.nebula_enrichment_api import *
from experts.common.RemoteAPIUtility import RemoteAPIUtility
from nebula_api.vlmapi import VLM_API
from nebula_api.atomic2020.comet_enrichment_api import *
from nebula_api.canonisation_api import CANON_API
import nebula_api.playground_api as pg_api

In [2]:
nre = NRE_API()
api = RemoteAPIUtility()
# vlm = VLM_API()
# ascore = CANON_API()

In [3]:
def flatten(lst): return [x for l in lst for x in l]

def compute_batch_scores(video_emb: torch.Tensor, texts: List[str], normalize=True, **kwargs) -> List[float]:    
    emb_batch = vlm.encode_text(texts, **kwargs)
    if type(emb_batch) == list:
        emb_batch = torch.stack(emb_batch,axis=0)
    if normalize:
        video_emb = (video_emb / video_emb.norm(2)).squeeze()           # This should be done in vlm
        # print("normalized video norm: {}".format(video_emb.norm(2)))
        n = (emb_batch * emb_batch).sum(axis=1).sqrt()
        emb_batch = emb_batch / n.unsqueeze(1).expand_as(emb_batch)
        # print("normalized text norms:")
        # for emb in emb_batch:
        #     print(emb.norm(2))                        
    return (video_emb.unsqueeze(0).expand_as(emb_batch)*emb_batch).sum(dim=1).cpu().numpy()


def compute_concat_score(image_emb: torch.Tensor, texts: List[str], join_on=',') -> float:
    combined_text = ""
    for t in [x.strip() for x in texts]:
        if t[-1]=='.':
            t = t[:-1]       
        t+=join_on
        t+=' '
        combined_text+=t
    print("Combined: "+combined_text)
    return torch.matmul(image_emb,mdmmt.encode_text(combined_text.strip()) )       

In [4]:
fusion_model="davinci:ft-personal:fusion-2022-03-29-21-07-19"
fusion_prompt_template="Original: {}\nCandidates: {}\n\n###\n\n"

def gpt_execute(prompt_template, *args, **kwargs):            
    prompt = prompt_template.format(*args)   
    response = openai.Completion.create(prompt=prompt, max_tokens=256, **kwargs)   
    return response

def gpt_fusion_ft(base, experts, **kwargs):
    rc = gpt_execute(fusion_prompt_template, base, '; '.join(experts), stop=["\n"], model=fusion_model, **kwargs)
    return [x['text'].strip() for x in rc['choices']]


def gpt_batch_fusion(base, all_expert_combs, **kwargs):    
    prompts = [fusion_prompt_template.format(base,'; '.join(exp)) for exp in all_expert_combs]
    rc = openai.Completion.create(prompt=prompts, max_tokens=256, stop=["\n"], model=fusion_model, **kwargs)
    return [x['text'].strip() for x in rc['choices']]
 

In [ ]:
MAX_PROMPT_NUM = 20

def gpt_process_fusion(base, all_experts, **kwargs):
    rc = []
    flattened_experts = flatten(all_experts.values())
    print("Total number of experts: {}".format(len(flattened_experts)))
    exp_combinations = [list(x) for x in list(itertools.combinations(flattened_experts,3)) + list(itertools.combinations(flattened_experts,2))]
    # exp_combinations = [list(x) for x in list(itertools.combinations(flattened_experts,2))]
    print("Total number of expert combinations: {}".format(len(exp_combinations)))
    chunked_experts=[exp_combinations[i:i + MAX_PROMPT_NUM] for i in range(0, len(exp_combinations), MAX_PROMPT_NUM)]
    for experts in chunked_experts:
        # print('\n\n--------------\n\n')
        # print(experts)
        rc.extend(gpt_batch_fusion(base,experts, **kwargs))
    return rc

def process_scene(doc, max_sentences=10, **kwargs):
    mid = doc['movie_id']
    elem = doc['scene_element']
    rc = nre.get_scene_from_collection(mid,elem,'s1_pipeline_results_phase2')
    if rc:
        print("Results already exist for {}/{}".format(mid,elem))
        return
    print("Going forward with {}/{}".format(mid,elem))
    done = False
    while not done:
        try:
            rc = gpt_process_fusion(doc['base'],doc['experts'], **kwargs)
            done=True
        except:
            print('Error, re-trying')
    rc_doc = {
        'movie_id': doc['movie_id'],
        'scene_element': doc['scene_element'],
        'combined_sentences': rc
    }
    query = "INSERT {} INTO s1_pipeline_results_phase2".format(rc_doc)
    cursor = nre.db.aql.execute(query)     


In [ ]:
query = 'FOR doc IN s1_lsmdc RETURN doc'
cursor = nre.db.aql.execute(query)
all_docs = list(cursor)

In [ ]:
process_scene(all_docs[1],n=2)